In [1]:
#pip install pytesseract opencv-python-headless
#pip install llama-cpp-python
#!brew install tesseract -- install tessarct in mac

In [2]:
import cv2
import pytesseract
from llama_cpp import Llama
import json

In [3]:
file_path = "/Users/sivaramakrishnan/Development/Donut/sample.png"
model_name = "google/gemma-2b-it"
model_file = "/Users/sivaramakrishnan/.cache/lm-studio/models/Mozilla/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf"

In [4]:
# Set the path to Tesseract executable (adjust this based on your installation)
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'

# Read the image (replace 'photo.jpg' with your image file path)
img = cv2.imread(file_path)

# Configuration for Tesseract (you can adjust this as needed)
config = ('-l eng --oem 1 --psm 3')

# Extract text from the image
extracted_text = pytesseract.image_to_string(img, config=config)
extracted_text

'Payslip\nZoonodle Inc\n\n21023 Pearson Point Road\nGateway Avenue\n\nDate of Joining : 2018-06-23 Employee name : Sally Harley\nPay Period : August 2021 Designation : Marketing Executive\nWorked Days 126 Department : Marketing\nEarnings Amount\nBasic 10000\nIncentive Pay 1000\nHouse Rent Allowance 400\nMeal Allowance 200\nTotal Earnings 11600\nDeductions Amount\nProvident Fund 1200\nProfesional Tax 500\nLoan 400\nTotal Deductions 2100\nNet Pay 9500\n9500\n\nNine Thousand Five Hundred\n\nEmployer Signature\n\nEmployee Signature\n\nThis is system generated payslip\n\n'

In [5]:
llm = Llama(model_path=model_file,
            n_gpu_layers=-1)

llama_model_loader: loaded meta data with 21 key-value pairs and 164 tensors from /Users/sivaramakrishnan/.cache/lm-studio/models/Mozilla/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attent

In [6]:
instruction = "<INST> extract data in json format with the document type </INST>\n" + extracted_text
response = llm(instruction, max_tokens=5000)
rtext = response['choices'][0]['text']
rtext


llama_print_timings:        load time =    1224.03 ms
llama_print_timings:      sample time =     456.04 ms /   236 runs   (    1.93 ms per token,   517.50 tokens per second)
llama_print_timings: prompt eval time =    1223.39 ms /   193 tokens (    6.34 ms per token,   157.76 tokens per second)
llama_print_timings:        eval time =   16025.60 ms /   235 runs   (   68.19 ms per token,    14.66 tokens per second)
llama_print_timings:       total time =   18276.59 ms /   428 tokens


'```json\n{\n  "document_type": "Payslip",\n  "company_name": "Zoonodle Inc",\n  "address": "21023 Pearson Point Road, Gateway Avenue",\n  "date_of_joining": "2018-06-23",\n  "employee_name": "Sally Harley",\n  "pay_period": "August 2021",\n  "designation": "Marketing Executive",\n  "worked_days": 126,\n  "earnings": {\n    "basic": 10000,\n    "incentive_pay": 1000,\n    "house_rent_allowance": 400,\n    "meal_allowance": 200\n  },\n  "deductions": {\n    "provident_fund": 1200,\n    "professional_tax": 500,\n    "loan": 400\n  },\n  "net_pay": 9500\n}\n```'

In [7]:
json_text = rtext[rtext.find("{"):rtext.rfind("}")+1]
json_text

'{\n  "document_type": "Payslip",\n  "company_name": "Zoonodle Inc",\n  "address": "21023 Pearson Point Road, Gateway Avenue",\n  "date_of_joining": "2018-06-23",\n  "employee_name": "Sally Harley",\n  "pay_period": "August 2021",\n  "designation": "Marketing Executive",\n  "worked_days": 126,\n  "earnings": {\n    "basic": 10000,\n    "incentive_pay": 1000,\n    "house_rent_allowance": 400,\n    "meal_allowance": 200\n  },\n  "deductions": {\n    "provident_fund": 1200,\n    "professional_tax": 500,\n    "loan": 400\n  },\n  "net_pay": 9500\n}'

In [8]:
json.loads(json_text)

{'document_type': 'Payslip',
 'company_name': 'Zoonodle Inc',
 'address': '21023 Pearson Point Road, Gateway Avenue',
 'date_of_joining': '2018-06-23',
 'employee_name': 'Sally Harley',
 'pay_period': 'August 2021',
 'designation': 'Marketing Executive',
 'worked_days': 126,
 'earnings': {'basic': 10000,
  'incentive_pay': 1000,
  'house_rent_allowance': 400,
  'meal_allowance': 200},
 'deductions': {'provident_fund': 1200, 'professional_tax': 500, 'loan': 400},
 'net_pay': 9500}